In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
ls ${FD_RES}/region

annotation_cCREs    annotation_motif      KS91_K562_ASTARRseq_peak_macs
annotation_gencode  annotation_remap2022


In [4]:
ls ${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs

KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.ccre_all.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.nuc.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.remap2022_nr.bed.gz
KS91_K562_hg38_ASTARRseq_Input.ccre_all.freq.tsv
KS91_K562_hg38_ASTARRseq_Input.remap2022_nr.tsv


In [5]:
ls ${FD_RES}/region/annotation_gencode

gencode.v42.basic.annotation.tss.bed


## Annotation with TSS

In [10]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_gencode_tss.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_gencode
FN_ANN=gencode.v42.basic.annotation.tss.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.gencode_v24_tss.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect -a ${FP_BED} -b ${FP_ANN} -wo -F 1.0 | gzip -c > ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29432948


In [11]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_gencode_tss.txt

Hostname:           x1-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-15-22+14:30:41

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_gencode/gencode.v42.basic.annotation.tss.bed

show first few lines of input
chr1	11868	11869	ENSG00000290825.1	.	+	DDX11L2
chr1	12009	12010	ENSG00000223972.6	.	+	DDX11L1
chr1	29570	29571	ENSG00000227232.5	.	-	WASH7P
chr1	17436	17437	ENSG00000278267.1	.	-	MIR6859-1
chr1	29553	29554	ENSG00000243485.5	.	+	MIR1302-2HG


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.gencode_v24_tss.bed.gz

show first few lines of outp